In [1]:
from osgeo import gdal, gdal_array, osr, ogr
import numpy as np
import rasterio
import pyproj
from affine import Affine
import pandas as pd
from shapely.geometry import Point
from geopandas import GeoDataFrame
import geopandas as gpd
from rasterio.features import shapes
import xarray

In [2]:
!pwd

/home/ubuntu/eda/flood-risk/data


In [3]:
%cd "/home/ubuntu/eda/data"

/home/ubuntu/eda/data


In [4]:
!pwd

/home/ubuntu/eda/data


In [5]:
# elevation data
dataset = gdal.Open('./elevation/DEM_LiDAR_1ft_2010_Improved_NYC_int.tif')
gt = dataset.GetGeoTransform()
band = dataset.GetRasterBand(1)
print(f'Elevation dataset origin: {(gt[0],gt[3])}')
print(f'Elevation dataset scale: {gt[1],gt[5]}')

Elevation dataset origin: (910719.3, 275160.675)
Elevation dataset scale: (1.0, -1.0)


In [6]:
dataset.RasterXSize

158100

In [7]:
dataset.RasterYSize

156100

In [9]:
array = gdal_array.BandReadAsArray(band)
print(array.shape)

(156100, 158100)


In [ ]:
# land cover data
lc_data = gdal.Open('./raster_data/NYC_2017_LiDAR_LandCover.img')
gt = lc_data.GetGeoTransform()
band = lc_data.GetRasterBand(1)
print(f'Land cover dataset origin: {(gt[0],gt[3])}')
print(f'Land cover dataset scale: {gt[1],gt[5]}')

In [15]:
# catch basin data
cb_data = ogr.Open('./DEPCatchbasins/DEPCATCHBASINS.shp')
layer = cb_data.GetLayer()
cb_ref = layer.GetExtent()
print(f'Catch basin dataset origin: {(cb_ref[0],cb_ref[3])}')
print(f'Catch basin dataset scale: (1.0, -1.0)')

Catch basin dataset origin: (913346.4203000069, 271705.03180000186)
Catch basin dataset scale: (1.0, -1.0)


In [7]:
#elevation data
elev_data = gdal.Open('./elevation/DEM_LiDAR_1ft_2010_Improved_NYC_int.tif')
elev_gt = elev_data.GetGeoTransform()
# print('Band count:', elev_data.RasterCount)
# print('Elev geotransf:', elev_geotransform)
#elev_band = elev_data.GetRasterBand(1)
print(f'Elevation dataset origin: {(elev_gt[0],elev_gt[3])}')
print(f'Elevation dataset scale: {elev_gt[1],elev_gt[5]}')

Elevation dataset origin: (910719.3, 275160.675)
Elevation dataset scale: (1.0, -1.0)


In [11]:
# lc_origin = (gt[0],gt[3])
# cb_origin = (cb_ref[0],cb_ref[3])
# elev_origin = (elev_gt[0], elev_gt[3])
# offset_ft_cb = (cb_origin[0]-lc_origin[0], cb_origin[1]-lc_origin[1])
# offset_pix_cb = (int(offset_ft_cb[0]*2),int(offset_ft_cb[1]*2))
# offset_ft_elev = (elev_origin[0]-lc_origin[0], elev_origin[1]-lc_origin[1])
# offset_pix_elev = (int(offset_ft_elev[0]*2),int(offset_ft_elev[1]*2))


# print('Land cover top left georef:',lc_origin)
# print('Catch basin top left georef:', cb_origin)
# print('Elevation top left georef:', elev_origin)
# print(f'CB is shifted by {offset_ft_cb} ft')
# print(f'Elev is shifted by {offset_ft_elev} ft')
# print(f'CB array top left offset: {offset_pix_cb} pixels (array elements)')
# print(f'Elev array top left offset: {offset_pix_elev} pixels (array elements)')

Land cover top left georef: (912286.93, 273618.3)
Catch basin top left georef: (913346.4203000069, 271705.03180000186)
Elevation top left georef: (910719.3, 275160.675)
CB is shifted by (1059.4903000068152, -1913.2681999981287) ft
Elev is shifted by (-1567.6300000000047, 1542.375) ft
CB array top left offset: (2118, -3826) pixels (array elements)
Elev array top left offset: (-3135, 3084) pixels (array elements)


In [7]:
!pwd

/home/ubuntu/eda/data


In [ ]:
## point : 40.931004, -74.057991
## offset from elevation origin (ft) : (57506, -3313)

In [ ]:
def get_coords_from_lat_long(lat, long, proj_from = pyproj.Proj(2263), proj_to = pyproj.Proj(proj = 'latlong')):
    transformer = pyproj.Transformer.from_proj(proj_to, proj_from)
    eastings, northings = transformer.transform(long, lat)
    x = int((eastings - geotransform[0])/geotransform[1]-0.5)
    y = int((northings - geotransform[3])/geotransform[5]-0.5)
    return x, y

In [ ]:
def conv_shp_to_raster(shp_fp, raster_fp, base_fp, no_data_val = 0, scale_factor = 1, offset_point = None, dtype = gdal.GDT_Byte):
    
    #get base dataset attributes
    base_data = gdal.Open(base_fp)
    x_dim = base_data.RasterXSize
    y_dim = base_data.RasterYSize
    base_gt = base_data.GetGeoTransform()

    #open shapefile, get layer and CRS
    layer = ogr.Open(shp_fp).GetLayer()
    source_crs = layer.GetSpatialRef().ExportToWkt()
    
    #compute offset
    if offset_point:
        x_offset, y_offset = get_coords_from_lat_long(np.array([offset_point[0]]),np.array([offset_point[1]]))
    else:
        x_offset = 0
        y_offset = 0
    
    x_res = 
        
    
    output_ds = gdal.GetDriverByName('GTiff').Create(raster_fp, x_dim, y_dim, 1, dtype)
    output_ds.SetGeoTransform(base_gt[0]+x_offset, 

In [6]:
%%time
# vector_fn = './DEPCatchbasins/DEPCATCHBASINS.shp'
raster_fn = './new_raster/catch_basin_1ft_raster.tif'
data = ogr.Open('./DEPCatchbasins/DEPCATCHBASINS.shp')
layer = data.GetLayer()
source_srs = layer.GetSpatialRef()
srs = source_srs.ExportToWkt()
x_min, x_max, y_min, y_max = layer.GetExtent()

# pixel_size = 2
pixel_size = 1
NoData_value = 0

# x_res = int(310844/4)
# y_res = int(314414/4)
x_res = int((310844-111877)/2)
y_res = int((314414-9711)/2)
target_ds = gdal.GetDriverByName('GTiff').Create(raster_fn, x_res, y_res, 1, gdal.GDT_Float32)
target_ds.SetGeoTransform((gt[0]+57506, pixel_size, 0, gt[3]-3313, 0, -pixel_size))
target_ds.SetProjection(srs)
band = target_ds.GetRasterBand(1)
band.SetNoDataValue(NoData_value)

# Rasterize
gdal.RasterizeLayer(target_ds, [1], layer, burn_values=[1])

# # Read as array
# array = band.ReadAsArray()
# print(array)
# print(array.shape)

CPU times: user 44 s, sys: 33 s, total: 1min 16s
Wall time: 1min 18s


0

In [6]:
310844*314414

97733705416

In [7]:
##array size
(310844-111877)/2*(314414-9711)/2

15156460450.25

In [11]:
array.min()

0.0

In [12]:
array.max()

1.0

In [10]:
sum(sum(array))

135004.0

In [27]:
dataset = gdal.Open('./new_raster/cb_raster.tif')
print(dataset.GetGeoTransform())

(912286.93, 2.0, 0.0, 273618.3, 0.0, -2.0)


In [ ]:
%%time
# vector_fn = './DEPCatchbasins/DEPCATCHBASINS.shp'
raster_fn = './new_raster/subway_entrance_1ft_raster.tif'

data = ogr.Open('./doitt_subway_entrances/DOITT_SUBWAY_ENTRANCE_04JAN2017.shp')
layer = data.GetLayer()
source_srs = layer.GetSpatialRef()
srs = source_srs.ExportToWkt()
x_min, x_max, y_min, y_max = layer.GetExtent()

# pixel_size = 2
pixel_size = 1
NoData_value = 0

# x_res = int(310844/4)
# y_res = int(314414/4)
x_res = int((310844-111877)/2)
y_res = int((314414-9711)/2)
target_ds = gdal.GetDriverByName('GTiff').Create(raster_fn, x_res, y_res, 1, gdal.GDT_Float32)
target_ds.SetGeoTransform((gt[0]+111877/2, pixel_size, 0, gt[3]-9711/2, 0, -pixel_size))
target_ds.SetProjection(srs)
band = target_ds.GetRasterBand(1)
band.SetNoDataValue(NoData_value)

# Rasterize
gdal.RasterizeLayer(target_ds, [1], layer, burn_values=[1])

# Read as array
array = band.ReadAsArray()
print(array)
print(array.shape)